In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from nltk.stem.porter import PorterStemmer
import nltk
import re, string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn import metrics

In [2]:
import pandas as pd
import io

df = pd.read_csv('train_150k.txt', sep='\t')
df = df[['content','annotation']]
print(df)

                                                  content  annotation
0       Starting  back at work today   Looks like it'l...           0
1       Sugar levels dropping... munchies setting in. ...           1
2           @karineb22 yeah!!! have a great summer break!           1
3       hannah montana was very good.  now going to re...           1
4       @Mayra326 aww, have fun!  I just had my 3D las...           1
...                                                   ...         ...
149980  Had a great night at Tabs but now I'm missing ...           0
149981       What a night at work.. Church in the morning           1
149982  this book is terrible  im used 2 reading all  ...           0
149983  @backstreetboys Updates? Any?  I miss your upd...           0
149984                            @lexiewohlfort I agree!           1

[149985 rows x 2 columns]


In [3]:
nltk.download('stopwords')
stop = stopwords.words('english')

regex = re.compile('[%s]' % re.escape(string.punctuation))

def test_re(s):
    return regex.sub('', s)

df ['content_without_stopwords'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df ['content_without_puncs'] = df['content_without_stopwords'].apply(lambda x: regex.sub('',x))
del df['content_without_stopwords']
del df['content']
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,annotation,content_without_puncs
0,0,Starting back work today Looks like itll raini...
1,1,Sugar levels dropping munchies setting in Luck...
2,1,karineb22 yeah great summer break
3,1,hannah montana good going read twilight
4,1,Mayra326 aww fun I 3D last week
...,...,...
149980,0,Had great night Tabs Im missing Chad Going bed
149981,1,What night work Church morning
149982,0,book terrible im used 2 reading make believes ...
149983,0,backstreetboys Updates Any I miss updates guys


In [4]:
#Stemming
porter_stemmer = PorterStemmer()
#punctuations
nltk.download('punkt')
tok_list = []
size = df.shape[0]

for i in range(size):
  word_data = df['content_without_puncs'][i]
  nltk_tokens = nltk.word_tokenize(word_data)
  final = ''
  for w in nltk_tokens:
    final = final + ' ' + porter_stemmer.stem(w)
  tok_list.append(final)

df['content_tokenize'] = tok_list
del df['content_without_puncs']
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,annotation,content_tokenize
0,0,start back work today look like itll rain nex...
1,1,sugar level drop munchi set in luckili i blac...
2,1,karineb22 yeah great summer break
3,1,hannah montana good go read twilight
4,1,mayra326 aww fun i 3d last week
...,...,...
149980,0,had great night tab im miss chad go bed
149981,1,what night work church morn
149982,0,book terribl im use 2 read make believ teenag...
149983,0,backstreetboy updat ani i miss updat guy


In [5]:
noNums = []
for i in range(len(df)):
  noNums.append(''.join([i for i in df['content_tokenize'][i] if not i.isdigit()]))

df['content'] = noNums
df

,annotation,content_tokenize,content
0,0,start back work today look like itll rain nex...,start back work today look like itll rain nex...
1,1,sugar level drop munchi set in luckili i blac...,sugar level drop munchi set in luckili i blac...
2,1,karineb22 yeah great summer break,karineb yeah great summer break
3,1,hannah montana good go read twilight,hannah montana good go read twilight
4,1,mayra326 aww fun i 3d last week,mayra aww fun i d last week
...,...,...,...
149980,0,had great night tab im miss chad go bed,had great night tab im miss chad go bed
149981,1,what night work church morn,what night work church morn
149982,0,book terribl im use 2 read make believ teenag...,book terribl im use read make believ teenag ...
149983,0,backstreetboy updat ani i miss updat guy,backstreetboy updat ani i miss updat guy


In [6]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True, sublinear_tf=True)
tfIdf = tfIdfVectorizer.fit_transform(df.content.tolist())
print(tfIdf)

  (0, 25055)	0.2170450490302833
  (0, 22143)	0.4094546791594786
  (0, 81075)	0.31486919275177944
  (0, 94322)	0.327529281160234
  (0, 55631)	0.4290434910816175
  (0, 67048)	0.22988424530861507
  (0, 68609)	0.2721112422934833
  (0, 116130)	0.2414070553037897
  (0, 125956)	0.2299073165137419
  (0, 8228)	0.2513183751049937
  (0, 108079)	0.30586761185000644
  (1, 37201)	0.3206581319091309
  (1, 116096)	0.41795495750243167
  (1, 18976)	0.254760040871419
  (1, 11591)	0.2583175157901944
  (1, 69606)	0.3272559833312871
  (1, 54159)	0.20747341307111136
  (1, 101890)	0.24213601765241063
  (1, 78801)	0.37910701405117186
  (1, 29431)	0.2489607444192468
  (1, 66640)	0.29291461069531405
  (1, 109749)	0.3009772453615421
  (2, 13558)	0.3971407786324098
  (2, 109887)	0.3521198020245151
  (2, 41063)	0.29172835623278703
  :	:
  (149981, 19488)	0.5987783224752943
  (149981, 77408)	0.4147440583152412
  (149981, 125956)	0.33846855316291014
  (149982, 117409)	0.3613609599433531
  (149982, 119726)	0.323647536

In [7]:
df2 = pd.DataFrame(tfIdf[2].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"]) #for second entry only(just to check if working)
df2 = df2.sort_values('TF-IDF', ascending=False)
print (df2.head(10))

                TF-IDF
karineb       0.730179
break         0.397141
summer        0.352120
yeah          0.316266
great         0.291728
partyuggghh   0.000000
partyughh     0.000000
paso          0.000000
paskalamonik  0.000000
partysleepov  0.000000


In [8]:
dfx = pd.DataFrame(tfIdf.toarray(), columns = tfIdfVectorizer.get_feature_names_out())
def display_scores(vectorizer, tfidf_result):
    scores = zip(vectorizer.get_feature_names_out(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    i=0
    for item in sorted_scores:
        print ("{0:50} Score: {1}".format(item[0], item[1]))
        i = i+1
        if (i > 25):
          break

#top 25 words
display_scores(tfIdfVectorizer, tfIdf)

im                                                 Score: 2577.0953030191854
go                                                 Score: 2252.2468572354537
day                                                Score: 1896.6507641532678
get                                                Score: 1773.3585776953341
work                                               Score: 1697.2953366719146
good                                               Score: 1658.918683089924
it                                                 Score: 1520.352000209271
love                                               Score: 1477.7652287715869
like                                               Score: 1405.446263653671
today                                              Score: 1343.3904638364693
got                                                Score: 1260.2563619959121
cant                                               Score: 1252.5732569720897
time                                               Score: 1220.3408145390354
th

In [ ]:
X=tfIdf.toarray()
y = np.array(df.annotation.tolist())
#Spltting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3113)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

unique_elements, counts_elements = np.unique(y_train, return_counts=True)
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
oversample = RandomOverSampler(sampling_strategy='not majority')
X_over, y_over = oversample.fit_resample(X_train, y_train)

In [ ]:
gnb = GaussianNB()
gnbmodel = gnb.fit(X_over, y_over)
y_pred = gnbmodel.predict(X_test)
print ("Score:", gnbmodel.score(X_test, y_test))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

In [ ]:
svc = SVC()
svcmodel = svc.fit(X_over, y_over)
y_pred = svcmodel.predict(X_test)
print ("Score:", svcmodel.score(X_test, y_test))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))